#### Введение

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов.

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C.

#### Реализация в Scikit-Learn

В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск.

В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В Scikit-Learn метрика AUC реализована функцией sklearn.metrics.roc_auc_score. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.

#### Материалы

- [Подробнее о логистической регрессии и предсказании вероятностей с ее помощью](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture05-linclass.pdf)
- [Подробнее о градиентах и градиентном спуске](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture02-linregr.pdf)

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as rms
from sklearn.metrics import roc_auc_score
from math import exp

#### 1 Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [19]:
data = pd.read_csv('data-logistic.csv', header=None)
x = data.loc[:, 1:]
y = data[0]

#### 2 Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [20]:
K = 0.1
ERROR = 1e-5

def sigma_y(i, w1, w2):
    return 1. / (1. + exp(-y[i] * (w1*x[1][i] + w2*x[2][i])))

def delta_for_w(w_index, w1, w2, C):
    addition = sum((
        y[i] * x[w_index][i] * (1. - sigma_y(i, w1, w2)) for i in np.arange(0, len(y))
    ))
    addition *= K / len(y)
    addition -= K * C * (w1 if w_index == 1 else w2)
    
    return addition

#### 3 Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [21]:
def gradient_regressor(C, iterations_remaining=10000):
    changed_w1, changed_w2 = 0., 0.
    while iterations_remaining:
        iterations_remaining -= 1
        w1, w2 = changed_w1, changed_w2
        changed_w1 = w1 + delta_for_w(1, w1, w2, C)
        changed_w2 = w2 + delta_for_w(2, w1, w2, C)
        if np.sqrt(rms([w1, w2], [changed_w1, changed_w2])) <= ERROR:
            break
    return changed_w1, changed_w2

def sigma(xi, w1, w2):
    return 1. / (1 + np.exp(-w1 * xi[1] - w2 * xi[2]))

#### 4 Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [22]:
w1, w2 = gradient_regressor(0.)
l2w1, l2w2 = gradient_regressor(10.)

print(w1, w2, l2w1, l2w2)

scores = x.apply(lambda xi: sigma(xi, w1, w2), axis=1)
l2scores = x.apply(lambda xi: sigma(xi, l2w1, l2w2), axis=1)

0.287683251283 0.0921020117342 0.0285587545462 0.0247801372497


#### 5 Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [23]:
auc_score = roc_auc_score(y, scores)
l2_auc_score = roc_auc_score(y, l2scores)

print(auc_score)
print(l2_auc_score)

f = open('submission.txt', 'w')
f.write(str(auc_score) + ' ' + str(l2_auc_score))
f.close()

0.926857142857
0.936285714286
